In [ ]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
import random
import pandas as pd
from tqdm import tqdm

tqdm.pandas()

In [ ]:
from datasets import load_dataset

ds = load_dataset("bdsaglam/musique", "answerable", split="train[:100]")
ds[0]

{'id': '2hop__482757_12019',
 'paragraphs': [{'idx': 0,
   'title': 'Pakistan Super League',
   'paragraph_text': 'Pakistan Super League (Urdu: پاکستان سپر لیگ \u202c \u200e; PSL) is a Twenty20 cricket league, founded in Lahore on 9 September 2015 with five teams and now comprises six teams. Instead of operating as an association of independently owned teams, the league is a single entity in which each franchise is owned and controlled by investors.',
   'is_supporting': False},
  {'idx': 1,
   'title': 'Serena Wilson',
   'paragraph_text': 'Serena Wilson (August 8, 1933 – June 17, 2007), often known just as "Serena", was a well-known dancer, choreographer, and teacher who helped popularize belly dance in the United States. Serena\'s work also helped legitimize the dance form and helped it to be perceived as more than burlesque or stripping. Serena danced in clubs in her younger years, opened her own studio, hosted her own television show, founded her own dance troupe, and was the auth

# Rerank Service Example

This notebook demonstrates how to use the reranker service via its Python client.

In [ ]:
from verifiers.rerank import RerankClient

In [ ]:
# Create a client instance
client = RerankClient(base_url="http://localhost:8931")

# Check health
health = client.health()
print("Health check:", health)

Health check: {'status': 'healthy'}


In [ ]:
i = random.choice(range(len(ds)))
example = ds[i]

docs = [p['paragraph_text'] for p in example['paragraphs']]

# Rerank the documents
query = example['question']

In [ ]:
response = client.rerank(
    query=query,
    documents=docs,
    top_n=3,
    model="tei",
    # model="bm25",
)
response

RerankResponse(results=[RerankResult(index=18, relevance_score=2.2525148391723633, document='Vietnam: The event was held in Ho Chi Minh City on April 29. Some 60 torchbearers carried the torch from the downtown Opera House to the Military Zone 7 Competition Hall stadium near Tan Son Nhat International Airport along an undisclosed route. Vietnam is involved in a territorial dispute with China (and other countries) for sovereignty of the Spratly and Paracel Islands; tensions have risen recently[when?] following reports that the Chinese government had established a county-level city named Sansha in the disputed territories, resulting in anti-Chinese demonstrations in December 2007 in Hanoi and Ho Chi Minh City. However to sustain its relationship with China the Vietnamese government has actively sought to head off protests during the torch relay, with Prime Minister Nguyễn Tấn Dũng warning government agencies that "hostile forces" may try to disrupt the torch relay.'), RerankResult(index=

In [ ]:
for result in response.results:
    paragraph = example['paragraphs'][result.index]
    print(paragraph['title'], paragraph['is_supporting'])

2008 Summer Olympics torch relay True
Ho Chi Minh City True
Torch Lake Township, Antrim County, Michigan False


In [ ]:
def make_retriever(model="tei", top_n=3):
    def retrieve(example):
        docs = [p['paragraph_text'] for p in example['paragraphs']]
        query = example['question']
        try:
            response = client.rerank(
                query=query,
                documents=docs,
                top_n=top_n,
                model=model,
            )
        except Exception as e:
            print(e)
            return []
        return [result.index for result in response.results]
    return retrieve

In [ ]:
def evaluate_retriever(df, retriever, top_n=3):
    df = df.copy()
    df['retrieved_doc_ids'] = df.apply(retriever, axis=1)
    df['supporting_doc_ids'] = df['paragraphs'].apply(lambda x: [p['idx'] for p in x if p['is_supporting']])
    df['recall'] = df.apply(lambda row: len(set(row['supporting_doc_ids']) & set(row['retrieved_doc_ids'])) / len(row['supporting_doc_ids']), axis=1)
    return df


In [ ]:
df = ds.to_pandas()

In [ ]:
# Alibaba-NLP/gte-multilingual-reranker-base
tei_retriever = make_retriever(model="tei", top_n=3)
tei_df = evaluate_retriever(df, tei_retriever)
tei_df['recall'].describe()

count    100.000000
mean       0.010000
std        0.070353
min        0.000000
25%        0.000000
50%        0.000000
75%        0.000000
max        0.500000
Name: recall, dtype: float64

In [ ]:
# BAAI/bge-reranker-large
tei2_retriever = make_retriever(model="tei", top_n=3)
tei2_df = evaluate_retriever(df, tei2_retriever)
tei2_df['recall'].describe()

API request failed with status 500: TEI endpoint returned error: 413 - {"error":"Input validation error: `inputs` must have less than 512 tokens. Given: 548","error_type":"Validation"}
API request failed with status 500: TEI endpoint returned error: 413 - {"error":"Input validation error: `inputs` must have less than 512 tokens. Given: 514","error_type":"Validation"}


count    100.000000
mean       0.740000
std        0.305505
min        0.000000
25%        0.500000
50%        1.000000
75%        1.000000
max        1.000000
Name: recall, dtype: float64

In [ ]:
# BAAI/bge-reranker-v2-m3
tei3_retriever = make_retriever(model="tei", top_n=3)
tei3_df = evaluate_retriever(df, tei3_retriever)
tei3_df['recall'].describe()

count    100.000000
mean       0.770000
std        0.279068
min        0.000000
25%        0.500000
50%        1.000000
75%        1.000000
max        1.000000
Name: recall, dtype: float64

In [ ]:
bm25_retriever = make_retriever(model="bm25", top_n=3)
bm25_df = evaluate_retriever(df, bm25_retriever)
bm25_df['recall'].describe()

count    100.00000
mean       0.45000
std        0.30567
min        0.00000
25%        0.50000
50%        0.50000
75%        0.50000
max        1.00000
Name: recall, dtype: float64

In [ ]:
flash_retriever = make_retriever(model="flashrank/ms-marco-MiniLM-L-12-v2", top_n=3)
flash_df = evaluate_retriever(df, flash_retriever)
flash_df['recall'].describe()

count    100.000000
mean       0.615000
std        0.273907
min        0.000000
25%        0.500000
50%        0.500000
75%        1.000000
max        1.000000
Name: recall, dtype: float64